In [2]:
pip install statsbombpy

     ---------------------------------------- 61.4/61.4 kB 1.6 MB/s eta 0:00:00
     ---------------------------------------- 57.0/57.0 kB 3.1 MB/s eta 0:00:00
     ---------------------------------------- 57.5/57.5 kB 3.1 MB/s eta 0:00:00
     ---------------------------------------- 60.8/60.8 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.3.0
    Uninstalling typing_extensions-4.3.0:
      Successfully uninstalled typing_extensions-4.3.0
  Attempting uninstall: attrs
    Found existing installation: attrs 21.4.0
    Uninstalling attrs-21.4.0:
      Successfully uninstalled attrs-21.4.0
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from statsbombpy import sb
import pandas as pd
from pandas import json_normalize
import numpy as np

In [4]:
sb.matches(competition_id=11, season_id=27)

c:\Users\conor.malone_skyspec\AppData\Local\Programs\Python\Python310\lib\site-packages\statsbombpy\api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


,match_id,match_date,kick_off,competition,season,home_team,away_team,home_score,away_score,match_status,...,last_updated_360,match_week,competition_stage,stadium,referee,home_managers,away_managers,data_version,shot_fidelity_version,xy_fidelity_version
0,3825848,2015-09-23,20:00:00.000,Spain - La Liga,2015/2016,Levante UD,Eibar,2,2,available,...,None,5,Regular Season,Estadio Ciudad de Valencia,NaN,Luis Lucas Alcaraz González,José Luis Mendilibar Etxebarria,1.1.0,2,2
1,3825895,2015-09-23,22:00:00.000,Spain - La Liga,2015/2016,Las Palmas,Sevilla,2,0,available,...,None,5,Regular Season,Estadio de Gran Canaria,Carlos del Cerro Grande,Francisco Herrera Lorenzo,Unai Emery Etxegoien,1.1.0,2,2
2,3825894,2016-05-01,18:15:00.000,Spain - La Liga,2015/2016,RC Deportivo La Coruña,Getafe,0,2,available,...,None,36,Regular Season,Estadio Abanca-Riazor,Carlos Clos Gómez,Víctor Sánchez del Amo,Juan Eduardo Esnáider Belén,1.1.0,2,2
3,3825855,2016-05-02,20:30:00.000,Spain - La Liga,2015/2016,Málaga,Levante UD,3,1,available,...,None,36,Regular Season,Estadio La Rosaleda,Iñaki Bikandi Garrido,Javier Gracia Carlos,Joan Francesc Ferrer Sicilia,1.1.0,2,2
4,3825908,2016-05-15,19:00:00.000,Spain - La Liga,2015/2016,Espanyol,Eibar,4,2,available,...,None,38,Regular Season,RCDE Stadium,Mario Melero López,Constantin Gâlcă,José Luis Mendilibar Etxebarria,1.1.0,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,266467,2015-09-23,20:00:00.000,Spain - La Liga,2015/2016,Celta Vigo,Barcelona,4,1,available,...,2021-06-13T16:17:31.694,5,Regular Season,Abanca-Balaídos,Alberto Undiano Mallenco,Manuel Eduardo Berizzo,Luis Enrique Martínez García,1.1.0,2,2
376,267273,2016-02-20,17:00:00.000,Spain - La Liga,2015/2016,Las Palmas,Barcelona,1,2,available,...,2021-06-13T16:17:31.694,25,Regular Season,Estadio de Gran Canaria,NaN,Enrique Setién Solar,Luis Enrique Martínez García,1.1.0,2,2
377,266490,2015-09-20,20:30:00.000,Spain - La Liga,2015/2016,Barcelona,Levante UD,4,1,available,...,2021-06-13T16:17:31.694,4,Regular Season,Spotify Camp Nou,David Fernández Borbalan,Luis Enrique Martínez García,Luis Lucas Alcaraz González,1.1.0,2,2
378,266254,2016-02-14,21:30:00.000,Spain - La Liga,2015/2016,Barcelona,Celta Vigo,6,1,available,...,2021-06-13T16:17:31.694,24,Regular Season,Spotify Camp Nou,NaN,Luis Enrique Martínez García,Manuel Eduardo Berizzo,1.1.0,2,2


In [62]:
lineup = sb.lineups(match_id=266254)
df = lineup["Barcelona"]

c:\Users\conor.malone_skyspec\AppData\Local\Programs\Python\Python310\lib\site-packages\statsbombpy\api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


In [21]:
events = sb.events(match_id=266254)

c:\Users\conor.malone_skyspec\AppData\Local\Programs\Python\Python310\lib\site-packages\statsbombpy\api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


In [63]:
def extract_last_position(position_list):
    if len(position_list) > 0:
        last_position = position_list[-1]['position']
    else:
        last_position = None
    return last_position

# Apply the function to the dataframe
df['last_position'] = df['positions'].apply(extract_last_position)

print(df)

    player_id                     player_name        player_nickname  \
0        4320   Neymar da Silva Santos Junior                 Neymar   
1        4324           Daniel Alves da Silva             Dani Alves   
2        4691      Claudio Andrés Bravo Muñoz          Claudio Bravo   
3        5203        Sergio Busquets i Burgos        Sergio Busquets   
4        5211                Jordi Alba Ramos             Jordi Alba   
5        5213           Gerard Piqué Bernabéu           Gerard Piqué   
6        5216            Andrés Iniesta Luján         Andrés Iniesta   
7        5246        Luis Alberto Suárez Díaz            Luis Suárez   
8        5470                    Ivan Rakitić                   None   
9        5503  Lionel Andrés Messi Cuccittini           Lionel Messi   
10       5506     Javier Alejandro Mascherano      Javier Mascherano   
11       6332                Thomas Vermaelen                   None   
12       6379          Sergi Roberto Carnicer          Sergi Rob

In [64]:
df["positions"].iloc[13]

[{'position_id': 2,
  'position': 'Right Back',
  'from': '60:20',
  'to': None,
  'from_period': 2,
  'to_period': None,
  'start_reason': 'Substitution - On (Tactical)',
  'end_reason': 'Final Whistle'}]

In [24]:
events["Time"] = events.minute.astype("str")+":"+events.second.astype("str")

In [16]:
events.columns

Index(['50_50', 'bad_behaviour_card', 'ball_receipt_outcome',
       'ball_recovery_offensive', 'ball_recovery_recovery_failure',
       'carry_end_location', 'clearance_aerial_won', 'clearance_body_part',
       'clearance_head', 'clearance_left_foot', 'clearance_right_foot',
       'counterpress', 'dribble_no_touch', 'dribble_nutmeg', 'dribble_outcome',
       'dribble_overrun', 'duel_outcome', 'duel_type', 'duration',
       'foul_committed_card', 'foul_committed_offensive',
       'foul_committed_type', 'foul_won_defensive', 'goalkeeper_body_part',
       'goalkeeper_end_location', 'goalkeeper_outcome', 'goalkeeper_position',
       'goalkeeper_technique', 'goalkeeper_type', 'half_end_early_video_end',
       'id', 'index', 'interception_outcome', 'location', 'match_id', 'minute',
       'off_camera', 'out', 'pass_aerial_won', 'pass_angle',
       'pass_assisted_shot_id', 'pass_body_part', 'pass_cross',
       'pass_cut_back', 'pass_end_location', 'pass_goal_assist', 'pass_height',

In [25]:
my_event = events[["player",'substitution_replacement',"Time","team"]][events.substitution_outcome=="Tactical"]
my_event

,player,substitution_replacement,Time,team
3533,Daniel Alves da Silva,Aleix Vidal Parreu,60:20,Barcelona
3534,Sergi Roberto Carnicer,Ivan Rakitić,60:48,Barcelona
3535,Pedro Pablo Hernández,Marcelo Alfonso Díaz Rojas,64:21,Celta Vigo
3536,John Guidetti,Dejan Dražić,76:56,Celta Vigo
3537,Andrés Iniesta Luján,Arda Turan,77:39,Barcelona
3538,Daniel Wass,Pape Cheikh Diop Gueye,82:21,Celta Vigo


In [65]:
results = pd.DataFrame()
for team in events.team.unique():
    df_event = events[(events.team==team) & (events.substitution_outcome=="Tactical")]
    df_lineup = lineup[team]
    df_lineup['last_position'] = df_lineup['positions'].apply(extract_last_position)
    merge_one = df_event[["minute","player",'substitution_replacement',"Time","team"]].merge(df_lineup[["player_name","last_position"]], left_on = "player", right_on = "player_name").rename(columns={"last_position":"starter_pos"})
    merge_two = merge_one[["minute","Time","team","player_name","starter_pos",'substitution_replacement']].merge(df_lineup[["player_name","last_position"]], right_on = "player_name", left_on = "substitution_replacement").rename(columns={"last_position":"sub_pos","player_name_x":"starter_name","substitution_replacement":"sub_name"}).drop("player_name_y",axis=1)
    results = pd.concat([results,merge_two],axis=0)

In [90]:
for index, row in results.iterrows():
    sub_time = row["minute"]
    before = events[["team","shot_statsbomb_xg"]][(events.minute<sub_time) &(events.minute>=(sub_time-10))].groupby("team",as_index=False).sum()#/events[["shot_statsbomb_xg"]][(events.minute<sub_time) &(events.minute>=(sub_time-10))].sum()
    before["xg_share"] = before["shot_statsbomb_xg"]/before["shot_statsbomb_xg"].sum()
    after = events[["team","shot_statsbomb_xg"]][(events.minute>=sub_time) &(events.minute<=(sub_time+10))].groupby("team",as_index=False).sum()
    after["xg_share"] = after["shot_statsbomb_xg"]/after["shot_statsbomb_xg"].sum()
    

# EXample 2: Percentage by lambda and DataFrame.apply() method.


In [91]:
before

,team,shot_statsbomb_xg,xg_share
0,Barcelona,2.332045,0.94692
1,Celta Vigo,0.130724,0.05308


In [92]:
after

,team,shot_statsbomb_xg,xg_share
0,Barcelona,0.502828,1.0
1,Celta Vigo,0.000000,0.0
